In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

YOUR_APP_CLIENT_ID = os.getenv("YOUR_APP_CLIENT_ID")
YOUR_APP_CLIENT_SECRET = os.getenv("YOUR_APP_CLIENT_SECRET")

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=YOUR_APP_CLIENT_ID,
    client_secret=YOUR_APP_CLIENT_SECRET,
    redirect_uri="http://127.0.0.1:8000/callback",
    scope="user-library-read,user-read-playback-state,user-read-playback-position,user-top-read,user-read-recently-played,user-follow-read",
))

results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(idx, track['artists'][0]['name'], " – ", track['name'])

In [ ]:
sp.current_user()

In [69]:
top_tracks = sp.current_user_top_tracks(limit=5, time_range='short_term')
tracks = top_tracks.get('items', [])

print("🎵 Your Top 5 Tracks (Short Term):\n")
for idx, track in enumerate(tracks, start=1):
    name = track['name']
    artists = ", ".join(artist['name'] for artist in track['artists'])
    album = track['album']['name']
    print(f"{idx}. {name} by {artists} (Album: {album})")

🎵 Your Top 5 Tracks (Short Term):

1. GUTS RAGE. (Berserk x HOLLY TEST Slowed) by snort., SAPFIR (Album: GUTS RAGE. (Berserk x HOLLY TEST Slowed))
2. ALORS KRUSH by RAIZHELL (Album: TRIBAL HELL)
3. MONTAGEM CANTANDO by CHASHKAKEFIRA (Album: MONTAGEM CANTANDO)
4. Sports car by Tate McRae (Album: So Close To What)
5. ROYALTY by FXRCE (Album: ROYALTY)


In [73]:
top_artists = sp.current_user_top_artists(limit=5, time_range='short_term')
artists = top_artists.get('items', [])

print("🎤 Your Top 5 Artists (Short Term):\n")
for idx, artist in enumerate(artists, start=1):
    name = artist['name']
    genres = ", ".join(artist.get('genres', [])) or "No genre info"
    print(f"{idx}. {name} (Genres: {genres})")

🎤 Your Top 5 Artists (Short Term):

1. CHASHKAKEFIRA (Genres: phonk, brazilian phonk)
2. Tate McRae (Genres: No genre info)
3. TEENWXVE (Genres: phonk, brazilian phonk)
4. DR MØB (Genres: phonk, brazilian phonk)
5. ARXMANE (Genres: phonk, brazilian phonk, rally house)


In [81]:
from collections import Counter

def get_top_genres_from_artists(sp, batch_count=5, batch_size=50, time_range='short_term'):
    all_genres = []
    total_artists = 0
    batch = 0

    for batch in range(batch_count):
        offset = batch * batch_size
        response = sp.current_user_top_artists(limit=batch_size, offset=offset, time_range=time_range)
        artists = response.get('items', [])
        batch_artist_count = len(artists)

        if batch_artist_count == 0:
            batch = batch - 1
            break

        total_artists += batch_artist_count

        for artist in artists:
            all_genres.extend(artist.get('genres', []))

    print(f"\nTotal of {total_artists} artists extracted in {batch + 1} batch(es).")

    # Count and display top 5 genres
    genre_counts = Counter(all_genres)
    top_genres = genre_counts.most_common(5)

    print(f"\n🎧 Your Top 5 Genres ({time_range}):\n")
    for idx, (genre, count) in enumerate(top_genres, start=1):
        print(f"{idx}. {genre} (Appears {count} time{'s' if count > 1 else ''})")

# Example usage:
get_top_genres_from_artists(sp, batch_count=5)


Total of 52 artists extracted in 2 batchs.

🎧 Your Top 5 Genres (short_term):

1. phonk (Appears 37 times)
2. brazilian phonk (Appears 26 times)
3. drift phonk (Appears 13 times)
4. anime (Appears 5 times)
5. anime rap (Appears 4 times)


In [85]:
from collections import Counter

def get_top_genres_from_tracks(sp, batch_count=5, batch_size=50, time_range='short_term'):
    all_genres = []
    total_tracks = 0

    for batch in range(batch_count):
        offset = batch * batch_size
        response = sp.current_user_top_tracks(limit=batch_size, offset=offset, time_range=time_range)
        tracks = response.get('items', [])
        batch_track_count = len(tracks)

        if batch_track_count == 0:
            batch = batch - 1
            break

        total_tracks += batch_track_count

        # Collect artist IDs from tracks
        artist_ids = set()
        for track in tracks:
            for artist in track.get('artists', []):
                artist_ids.add(artist['id'])

        # Fetch artist details to get genres
        # Spotify API lets you get up to 50 artists in one request
        artist_ids = list(artist_ids)
        for i in range(0, len(artist_ids), 50):
            batch_artist_ids = artist_ids[i:i+50]
            artists_response = sp.artists(batch_artist_ids)
            artists = artists_response.get('artists', [])
            for artist in artists:
                all_genres.extend(artist.get('genres', []))

    print(f"\nTotal of {total_tracks} tracks processed in {batch + 1} batch(es).")

    # Count and display top 5 genres
    genre_counts = Counter(all_genres)
    top_genres = genre_counts.most_common(5)

    print(f"\n🎧 Your Top 5 Genres (from tracks, {time_range}):\n")
    for idx, (genre, count) in enumerate(top_genres, start=1):
        print(f"{idx}. {genre} (Appears {count} time{'s' if count > 1 else ''})")

get_top_genres_from_tracks(sp, batch_count=20)


Total of 607 tracks processed in 13 batch(es).

🎧 Your Top 5 Genres (from tracks, short_term):

1. phonk (Appears 521 times)
2. brazilian phonk (Appears 350 times)
3. drift phonk (Appears 156 times)
4. anime rap (Appears 74 times)
5. anime (Appears 48 times)
